In [1]:
import mysql.connector

cnx = mysql.connector.connect(user='a20192340', password='changeme',
                              host='vsrv01.inesctec.pt',
                              database='finance')

import pandas as pd
account = pd.read_sql("SELECT * FROM account", cnx);
card_dev = pd.read_sql("SELECT * FROM card_dev", cnx);
client = pd.read_sql("SELECT * FROM client", cnx);
disp = pd.read_sql("SELECT * FROM disp", cnx);
district = pd.read_sql("SELECT * FROM district", cnx);
loan_dev = pd.read_sql("SELECT * FROM loan_dev", cnx);
trans_dev = pd.read_sql("SELECT * FROM trans_dev", cnx);

pd.set_option('display.expand_frame_repr', False)

cnx.close()

# Tabela account

In [2]:
account.head()

,account_id,district_id,frequency,date
0,576,55,monthly issuance,930101
1,3818,74,monthly issuance,930101
2,704,55,monthly issuance,930101
3,2378,16,monthly issuance,930101
4,2632,24,monthly issuance,930102


In [3]:
account.shape

(4500, 4)

In [4]:
account.district_id.shape

(4500,)

In [5]:
account.frequency.value_counts()

monthly issuance              4167
weekly issuance                240
issuance after transaction      93
Name: frequency, dtype: int64

In [6]:
#find missings
account.isnull().sum()[account.isnull().sum()>0]

Series([], dtype: int64)

In [7]:
account.dtypes

account_id      int64
district_id     int64
frequency      object
date            int64
dtype: object

In [8]:
for col in account.columns:
    if account[col].dtype=='object':
        print(col, ': ', account[col].unique(), '\n')

frequency :  ['monthly issuance' 'issuance after transaction' 'weekly issuance'] 



In [9]:
#tratar formato
account['ac_date']=pd.to_datetime(account['date'], format='%y%m%d',errors='coerce')

account.drop(['date'],axis=1, inplace=True)

account.ac_date.max()

Timestamp('1997-12-29 00:00:00')

# Tabela card_dev

In [10]:
card_dev.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,931107
1,104,588,classic,940119
2,747,4915,classic,940205
3,70,439,classic,940208
4,577,3687,classic,940215


In [11]:
card_dev.shape

(177, 4)

In [12]:
card_dev.type.value_counts()

classic    127
junior      41
gold         9
Name: type, dtype: int64

In [13]:
#find missings
card_dev.isnull().sum()[card_dev.isnull().sum()>0]

Series([], dtype: int64)

In [14]:
for col in card_dev.columns:
    if card_dev[col].dtype=='object':
        print(col, ': ', card_dev[col].unique(), '\n')

type :  ['classic' 'junior' 'gold'] 



In [15]:
#tratar formato
card_dev['issued_date']=pd.to_datetime(card_dev['issued'], format='%y%m%d',errors='coerce')

card_dev.drop(['issued'],axis=1, inplace=True)

card_dev.issued_date.max()

Timestamp('1996-12-31 00:00:00')

# Tabela client

In [16]:
client.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


In [17]:
client.shape

(5369, 3)

In [18]:
#find missings
client.isnull().sum()[client.isnull().sum()>0]

Series([], dtype: int64)

In [19]:
client.loc[client.birth_number%10000>5000,'sex']='F'
client.loc[client.birth_number%10000<5000,'sex']='M'

In [20]:
client.sex.value_counts()

M    2724
F    2645
Name: sex, dtype: int64

In [21]:
client.loc[client.birth_number%10000>5000,'birth']=client.loc[client.birth_number%10000>5000,'birth_number']-5000
client.loc[client.birth_number%10000<5000,'birth']=client.loc[client.birth_number%10000<5000,'birth_number']

In [22]:
#tratar formato
client['birth_date']=pd.to_datetime(client['birth'], format='%y%m%d',errors='coerce')

client.drop(['birth','birth_number'],axis=1, inplace=True)

In [23]:
#problema: máximo ano superior a 2000
#solucao: vou retirar 100 anos se ano sup 2000
client.birth_date.max()

Timestamp('2068-12-30 00:00:00')

In [24]:
client.loc[pd.DatetimeIndex(client.birth_date).year>2000,'birth_date']-=pd.DateOffset(years=100)

In [25]:
client.birth_date.max()

Timestamp('1987-09-27 00:00:00')

In [26]:
client.head()

,client_id,district_id,sex,birth_date
0,1,18,F,1970-12-13
1,2,1,M,1945-02-04
2,3,1,F,1940-10-09
3,4,5,M,1956-12-01
4,5,5,F,1960-07-03


# Tabela disp

In [27]:
disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [28]:
disp.shape

(5369, 4)

In [29]:
disp.type.value_counts()

OWNER        4500
DISPONENT     869
Name: type, dtype: int64

In [30]:
#find missings
disp.isnull().sum()[disp.isnull().sum()>0]

Series([], dtype: int64)

In [31]:
for col in disp.columns:
    if disp[col].dtype=='object':
        print(col, ': ', disp[col].unique(), '\n')

type :  ['OWNER' 'DISPONENT'] 



# Tabela district

In [32]:
district.head()

,code,name,region,no_inhabitants,no_municipalities_with_inhabitants_499,no_municipalities_inhabitants_500_1999,no_municipalities_with_inhabitants_2000_9999,no_municipalities_with_inhabitants_10000,no_cities,ratio_urban_inhabitants,average_salary,unemploymant_rate_95,unemploymant_rate_96,no_enterpreneurs_per_1000_inhabitants,no_commited_crimes_95,no_commited_crimes_96
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [33]:
district.columns

Index(['code', 'name', 'region', 'no_inhabitants',
       'no_municipalities_with_inhabitants_499',
       'no_municipalities_inhabitants_500_1999',
       'no_municipalities_with_inhabitants_2000_9999',
       'no_municipalities_with_inhabitants_10000', 'no_cities',
       'ratio_urban_inhabitants', 'average_salary', 'unemploymant_rate_95',
       'unemploymant_rate_96', 'no_enterpreneurs_per_1000_inhabitants',
       'no_commited_crimes_95', 'no_commited_crimes_96'],
      dtype='object')

In [34]:
district.shape

(77, 16)

In [35]:
#find missings
district.isnull().sum()[district.isnull().sum()>0]

Series([], dtype: int64)

In [36]:
for col in district.columns:
    if district[col].dtype=='object':
        print(col, ': ', district[col].unique(), '\n')

name :  ['Hl.m. Praha' 'Benesov' 'Beroun' 'Kladno' 'Kolin' 'Kutna Hora' 'Melnik'
 'Mlada Boleslav' 'Nymburk' 'Praha - vychod' 'Praha - zapad' 'Pribram'
 'Rakovnik' 'Ceske Budejovice' 'Cesky Krumlov' 'Jindrichuv Hradec'
 'Pelhrimov' 'Pisek' 'Prachatice' 'Strakonice' 'Tabor' 'Domazlice' 'Cheb'
 'Karlovy Vary' 'Klatovy' 'Plzen - mesto' 'Plzen - jih' 'Plzen - sever'
 'Rokycany' 'Sokolov' 'Tachov' 'Ceska Lipa' 'Decin' 'Chomutov'
 'Jablonec n. Nisou' 'Liberec' 'Litomerice' 'Louny' 'Most' 'Teplice'
 'Usti nad Labem' 'Havlickuv Brod' 'Hradec Kralove' 'Chrudim' 'Jicin'
 'Nachod' 'Pardubice' 'Rychnov nad Kneznou' 'Semily' 'Svitavy' 'Trutnov'
 'Usti nad Orlici' 'Blansko' 'Brno - mesto' 'Brno - venkov' 'Breclav'
 'Hodonin' 'Jihlava' 'Kromeriz' 'Prostejov' 'Trebic' 'Uherske Hradiste'
 'Vyskov' 'Zlin' 'Znojmo' 'Zdar nad Sazavou' 'Bruntal' 'Frydek - Mistek'
 'Jesenik' 'Karvina' 'Novy Jicin' 'Olomouc' 'Opava' 'Ostrava - mesto'
 'Prerov' 'Sumperk' 'Vsetin'] 

region :  ['Prague' 'central Bohemia' 'sout

In [37]:
#tratar formato e '?'
district['no_commited_crimes_95']=pd.to_numeric(district['no_commited_crimes_95'], errors='coerce')
district['unemploymant_rate_95']=pd.to_numeric(district['unemploymant_rate_95'], errors='coerce')

### Missings

In [38]:
district.isnull().sum()[district.isnull().sum()>0]

unemploymant_rate_95     1
no_commited_crimes_95    1
dtype: int64

# Tabela loan_dev

In [39]:
loan_dev.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1


In [40]:
loan_dev.shape

(328, 7)

In [41]:
loan_dev.status.value_counts()

 1    282
-1     46
Name: status, dtype: int64

In [42]:
#tratar status (acreditando que -1 é risco e 1 é sem risco)
mapp={1:0,-1:1}
loan_dev['status']=loan_dev['status'].replace(mapp)

In [43]:
#find missings
loan_dev.isnull().sum()[loan_dev.isnull().sum()>0]

Series([], dtype: int64)

In [44]:
#tratar formato
loan_dev['loan_date']=pd.to_datetime(loan_dev['date'], format='%y%m%d',errors='coerce')

loan_dev.drop(['date'],axis=1,inplace=True)

loan_dev['loan_date'].max()

Timestamp('1996-12-27 00:00:00')

# Tabela trans_dev

In [45]:
trans_dev.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1548749,5270,930113,credit,credit in cash,800.0,800.0,,,NaN
1,1548750,5270,930114,credit,collection from another bank,44749.0,45549.0,,IJ,80269753.0
2,3393738,11265,930114,credit,credit in cash,1000.0,1000.0,,,NaN
3,3122924,10364,930117,credit,credit in cash,1100.0,1100.0,,,NaN
4,1121963,3834,930119,credit,credit in cash,700.0,700.0,,,NaN


In [46]:
trans_dev.shape

(396685, 10)

In [47]:
trans_dev.account_id.unique().shape

(3370,)

In [48]:
trans_dev.k_symbol.value_counts()

                                         185244
interest credited                         70761
payment for statement                     58377
household                                 42839
                                          19065
old-age pension                           13502
insurrance payment                         6592
sanction interest if negative balance       305
Name: k_symbol, dtype: int64

### Missings

In [49]:
#find missings
trans_dev.isnull().sum()[trans_dev.isnull().sum()>0]

account    294456
dtype: int64

In [50]:
#bank e account são preenchidos juntos
trans_dev.loc[trans_dev.bank!=''].head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
1,1548750,5270,930114,credit,collection from another bank,44749.0,45549.0,,IJ,80269753.0
16,541683,1843,930207,credit,collection from another bank,17632.0,18632.0,,QR,76294219.0
23,237651,813,930211,credit,collection from another bank,12154.0,13254.0,,QR,19537103.0
26,1736609,5891,930212,credit,collection from another bank,32594.0,33494.0,,CD,51054984.0
32,1548751,5270,930214,credit,collection from another bank,44749.0,80808.2,,IJ,80269753.0


In [51]:
for col in trans_dev.columns:
    if trans_dev[col].dtype=='object':
        print(col, ': ', trans_dev[col].unique(), '\n')

type :  ['credit' 'withdrawal' 'withdrawal in cash'] 

operation :  ['credit in cash' 'collection from another bank' '' 'withdrawal in cash'
 'remittance to another bank' 'credit card withdrawal'] 

k_symbol :  ['' 'interest credited' ' ' 'household' 'payment for statement'
 'insurrance payment' 'sanction interest if negative balance'
 'old-age pension'] 

bank :  ['' 'IJ' 'QR' 'CD' 'KL' 'GH' 'ST' 'YZ' 'UV' 'MN' 'OP' 'AB' 'EF' 'WX'] 



In [52]:
# cuidado com '' ' '
map1={'':'Missing',' ':'Missing'}
trans_dev['operation']=trans_dev['operation'].replace(map1)
trans_dev['k_symbol']=trans_dev['k_symbol'].replace(map1)
trans_dev['bank']=trans_dev['bank'].replace(map1)

In [53]:
trans_dev['k_symbol'].unique()

array(['Missing', 'interest credited', 'household',
       'payment for statement', 'insurrance payment',
       'sanction interest if negative balance', 'old-age pension'],
      dtype=object)

In [54]:
#tratar formato
trans_dev['trans_date']=pd.to_datetime(trans_dev['date'], format='%y%m%d',errors='coerce')

trans_dev.drop(['date'],axis=1,inplace=True)

trans_dev['trans_date'].max()

Timestamp('1996-12-31 00:00:00')

In [55]:
#tratar categoria diferente extra ao README
trans_dev.loc[trans_dev.type=='withdrawal in cash','type']='withdrawal'

## Guardar datasets em zip

In [59]:
account.to_parquet('account.gzip',compression='gzip') 
card_dev.to_parquet('card_dev.gzip',compression='gzip') 
client.to_parquet('client.gzip',compression='gzip') 
disp.to_parquet('disp.gzip',compression='gzip') 
district.to_parquet('district.gzip',compression='gzip') 
loan_dev.to_parquet('loan_dev.gzip',compression='gzip') 
trans_dev.to_parquet('trans_dev.gzip',compression='gzip') 